In [30]:
import pandas as pd
import numpy as np

In [31]:
# read csv in to file
credits = pd.read_csv("credits.csv")
movies = pd.read_csv("score_processing.csv")

In [32]:
# create a director,actor dictionary with name as key, vlaue in the form [list of movie, number of movies acted ,average rating] and output as a new csv

MOVIE_ID = 0
MOVIE_COUNT = 1
SUM_MOVIE_SCORE = 2
AVG_SCORE = 3

actors_dict = {}
directors_dict = {}
movies_dict = {}

for index, data in movies.iterrows():
    if(data["id"] not in movies_dict.keys()):
        movies_dict[data["id"]] = data["weighted_average_popularity"]

for index, data in credits.iterrows():
    if(data["id"][1] == "s") :
        continue
    if(data["role"] == "ACTOR"):
        if(data["name"] not in actors_dict.keys()):
            actors_dict[data["name"]] = [[data["id"]],1,movies_dict[data["id"]],0]
        else:
            actors_dict[data["name"]][MOVIE_COUNT] += 1
            actors_dict[data["name"]][SUM_MOVIE_SCORE] += movies_dict[data["id"]]
            actors_dict[data["name"]][MOVIE_ID].append(data["id"])
    else:
        if(data["name"] not in directors_dict.keys()):
            directors_dict[data["name"]] = [[data["id"]],1,movies_dict[data["id"]],0]
        else:
            directors_dict[data["name"]][MOVIE_COUNT] += 1
            directors_dict[data["name"]][SUM_MOVIE_SCORE] += movies_dict[data["id"]]
            directors_dict[data["name"]][MOVIE_ID].append(data["id"])
for values in actors_dict.values():
    values[AVG_SCORE] = (values[SUM_MOVIE_SCORE]) / (values[MOVIE_COUNT])
for values in directors_dict.values():
    values[AVG_SCORE] = (values[SUM_MOVIE_SCORE]) / (values[MOVIE_COUNT])


In [33]:
# Put all the data into csv file
act_df = pd.DataFrame.from_dict(actors_dict)
act_df = act_df.transpose()
# add title to the data
column_titles = ['movie acted', 'num_of_movie',"sum_score" ,"avg_score"]  # Replace with desired column titles
act_df.columns = column_titles
act_df.index.name = "name"
act_df.to_csv("actor.csv")

#do exact same thing for director
dir_df = pd.DataFrame.from_dict(directors_dict)
dir_df = dir_df.transpose()
# add title to the data
column_titles = ['movie acted', 'num_of_movie',"sum_score" ,"avg_score"]  # Replace with desired column titles
dir_df.columns = column_titles
dir_df.index.name = "name"
dir_df.to_csv("director.csv")

calculate weighted score use max-min normalize method to adjust all the valuje from imdb and tmdb then apply weight to each of them

In [34]:
movies = pd.read_csv("pre-processed_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'pre-processed_data.csv'

In [ ]:
def normalize(imput_list):
    movies_array = np.array(imput_list)
    # Calculate min and max values
    min_val = np.min(movies_array)
    max_val = np.max(movies_array)

    # Perform Max-Min normalization
    normalized_data = (movies_array - min_val) / (max_val - min_val)
    return normalized_data

In [ ]:
nor_imdb_score = normalize(movies["imdb_score"])
nor_tmdb_score = normalize(movies["tmdb_score"])

In [ ]:
def log(data):
    return np.log1p(data)

In [ ]:
log_imdb_vote = log(movies["imdb_votes"])
log_tmdb_popularity = log(movies["tmdb_popularity"])

In [ ]:
movies["nor_imdb_score"] = nor_imdb_score
movies["log_imdb_vote"] = log_imdb_vote
movies["log_tmdb_popularity"] = log_tmdb_popularity
movies["nor_tmdb_score"] = nor_tmdb_score

#movies["weight_average_score"] = nor_imdb_score*3.5 + nor_imdb_vote*1.5 + nor_tmdb_popularity*1.5 + nor_tmdb_score*3.5
#movies.to_csv("wight_average_score.csv", index=False)
movies.to_csv("normalized.csv", index = False)

calculate popularity of an actor base on the movie and number of movies they acted 